In [1]:
import sys
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import datetime
from tqdm import tqdm
import GPy
from collections import defaultdict
from pathlib import Path
import seaborn as sns
import scipy.stats as stats
from matplotlib.colors import ListedColormap
import warnings
import time
from itertools import product
from joblib import Parallel, delayed
from operator import itemgetter
import sncosmo
import shutil

mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.autolayout'] = True
# mpl.rcParams['figure.dpi'] = 300
# mpl.rcParams['axes.spines.right'] = False
# mpl.rcParams['axes.spines.top'] = False


plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})


def utkarshGrid():
    plt.minorticks_on()
    plt.grid(color='grey',
             which='minor',
             linestyle=":",
             linewidth='0.1',
             )
    plt.grid(color='black',
             which='major',
             linestyle=":",
             linewidth='0.1',
             )

In [2]:
path = os.path.dirname(os.getcwd())
os.chdir(path)
from Emulator.Classes.AllData import AllData
from Emulator.Classes.LightCurve import utkarshGrid, LightCurve
from Emulator.Classes.GP import GP
from Emulator.Classes.GP2D import GP2D
from Emulator.Classes.GP5D import GP5D
os.chdir(os.getcwd() + "/Emulator")
os.getcwd()

'/Users/utkarsh/PycharmProjects/KilonovaeLightCurves/Emulator'

In [3]:
curr_wv = np.arange(100, 3600, 10)
set_skip_factor = None
gpSplit = 1

In [4]:
mejdyn = 0.01
mejwind =  0.13
phi = 45
iobs = 4


gp = GP5D("Classes/reference.csv")
gp.split = gpSplit
gp.emulator = "start"
gp.cross_validation = (mejdyn, mejwind, phi, iobs)
gp.set_wv_range(curr_wv)
gp.n_comp = 25
time_shape = None
# gp.save_training_data(time_trauncation = time_shape) # Which wv do you want to extract.  
gp.save_pca_components(skip_factor = set_skip_factor)
gp.setXY_cross_validation(mejdyn, mejwind, phi, iobs)
fitting_kernel = GPy.kern.RBF(input_dim=4, variance = 1, lengthscale=1, ARD = True)
decay_kernel = GPy.kern.Linear(input_dim=4, ARD = True)
gp.kernel = fitting_kernel * decay_kernel
gp.model = GPy.models.GPRegression(gp.X,gp.Y,gp.kernel)
# gp.model.Gaussian_noise.fix(10)
# gp.model.mul.rbf.variance.fix(10)
print(gp.model)
t0 = time.time()
print(f"[STATUS] Optimizing...")
gp.model.optimize_restarts(parallel = True, robust = True, num_restarts = 3)
print(f"[STATUS] {round(time.time() - t0, 2)}s taken to optimize.")
# gp.model.optimize(messages = True)
# display(gp.model.rbf.lengthscale)
display(gp.model)
display(gp.model.mul.rbf.lengthscale)
t1 = time.time()
gp.model_predict_cross_validation(include_like = True) # Save cross validation
print(f"[STATUS] {round(time.time() - t1, 2)}s taken to predict single item.")
gp.model_predict(include_like = True)
print(f"[STATUS] {round(time.time() - t1, 2)}s taken to predict every item.")
t2 = time.time()
gp.save_trained_data()
print(f"[STATUS] {round(time.time() - t2, 2)}s taken to save data.")
gp.ComputeDifferenceFlux()
gp.plot_difference_histogram(binning = 350)
plt.show()

[STATUS] Setting X, Y components for 5D Model.

Name : GP regression
Objective : 12598299.710575594
Number of Parameters : 10
Number of Optimization Parameters : 10
Updates : True
Parameters:
  GP_regression.           |  value  |  constraints  |  priors
  mul.rbf.variance         |    1.0  |      +ve      |        
  mul.rbf.lengthscale      |   (4,)  |      +ve      |        
  mul.linear.variances     |   (4,)  |      +ve      |        
  Gaussian_noise.variance  |    1.0  |      +ve      |        
[STATUS] Optimizing...
Optimization restart 1/3, f = 196727.2653693051
Optimization restart 2/3, f = 217047.10414014218
Optimization restart 3/3, f = 197329.51706910538
[STATUS] 108.31s taken to optimize.


GP_regression.,value,constraints,priors
mul.rbf.variance,43.75172563561967,+ve,
mul.rbf.lengthscale,"(4,)",+ve,
mul.linear.variances,"(4,)",+ve,
Gaussian_noise.variance,49.13869090074636,+ve,


index,GP_regression.mul.rbf.lengthscale,constraints,priors
[0],0.12553082,+ve,
[1],0.10146233,+ve,
[2],0.11547802,+ve,
[3],38.51148996,+ve,


[STATUS] Predicting X and Y with trained emulator.
[STATUS] 0.01s taken to predict single item.
[STATUS] Predicting X and Y with trained emulator.
[STATUS] 1.7s taken to predict every item.


ValueError: operands could not be broadcast together with shapes (35000,2154) (2155,) (35000,2154) 

In [ ]:
gp.get_flux(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs, time_desired = 2, wv_desired = 950)
gp.plot_filters(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs)
gp.overplot_time(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs, wv_desired = 950)
gp.overplot_wavelength(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs, time_desired = 2)
gp.plot_emulator_errors()
plt.show()

In [ ]:
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
self = gp
# col_name="coolwarm_r"
col_name="plasma"
t = self._t_helper()
untrained = np.load(f"data/pca/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
untrained = self._untrained_helper(untrained)
trained = np.load(f"data/pcaTrained/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
filters = ["sdss::u", "sdss::g", "sdss::r", "sdss::i", "sdss::z",
           "swope2::y", "swope2::J", "swope2::H"]

colors = sns.color_palette(col_name, len(filters))[::-1]

plt.figure(dpi=1000, figsize=(6, 2.5))
for i in range(len(filters)):
    source = sncosmo.TimeSeriesSource(t, self.wv_range * 10, 10 ** trained)
    source2 = sncosmo.TimeSeriesSource(t, self.wv_range * 10, 10 ** untrained)
    m = source.bandmag(filters[i], "ab", t)
    m2 = source2.bandmag(filters[i], "ab", t)
    plt.plot(t, m, color=colors[i], zorder=1, alpha = 0.7, linewidth = 2)
    plt.plot(t, m2, linestyle="dashed", alpha=0.7, color=colors[i], zorder=2, linewidth = 2)

plt.plot(t, np.ones(t.shape), linestyle = "dashed", color = "white", label = "Truth", alpha = 1)
plt.plot(t, np.ones(t.shape), linestyle = "solid", color = "white", label = "Simulation", alpha = 1)

cmap = plt.get_cmap(col_name,len(filters))
norm = mpl.colors.Normalize(vmin=0,vmax=2)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ticks=np.linspace(0,2,len(filters)), boundaries=np.arange(-0.05,2.1,.1))
cbar.ax.set_yticklabels([item[-1] for item in filters][::-1])
cbar.set_label('Photometric Band', rotation=270, labelpad=15)
# cbar.set_label('Photometric Band', labelpad=-40, y=1.05, rotation=0)


plt.gca().set_facecolor('black')
l = plt.legend(prop ={'weight':'bold'}, loc = "upper right")
for text in l.get_texts():
    text.set_color("white")

# split_arrow = 5
# plt.vlines(x = split_arrow, ymin = -2, ymax = -3, color = "white", alpha = 0.9)
# plt.annotate(text = "Emulator A", xy  = (1, -1.25), color = "white", fontsize = 9)
# plt.annotate(text = "Emulator B", xy  = (11, -1.25), color = "white", fontsize = 9)
# plt.annotate(text='', xy=(split_arrow,-2.5), xytext=(0,-2.5), arrowprops=dict(arrowstyle='<->', color='white'), color = "white")
# plt.annotate(text='', xy=(20,-2.5), xytext=(split_arrow,-2.5), arrowprops=dict(arrowstyle='<->', color='white'), color = "white")

utkarshGrid()
plt.ylim(-17, 0)
plt.xlim(-0.1, 20)
plt.gca().invert_yaxis()
plt.xlabel("Time (days)")
plt.ylabel("Flux (Magnitude)")
# plt.title(f"Magnitude plot as specified filters", fontsize = 16)
plt.show()

In [ ]:
self = gp
t = self._t_helper()
diff = np.zeros(t.shape)
counter = 0
eps = 1e-18

for index, row, in tqdm(self.reference.iterrows(), total=196):
    for viewing_angle in self.iobs_range:
        mejdyn = row.mejdyn
        mejwind = row.mejwind
        phi = row.phi
        iobs = viewing_angle

        try:
            untrained = np.load(f"data/pca/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
            trained = np.load(f"data/pcaTrained/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
            untrained = self._untrained_helper(untrained)
        except:
            continue

        # diff = diff + np.sum(np.abs((trained - untrained) / untrained), axis = 1)/self.num_wv
        diff = diff + 2.5 * np.sum(np.abs((trained - untrained)), axis=1) / self.num_wv
        counter += 1

diff = diff / counter  # Divide each wavelength and each training point and viewing angle

plt.figure(dpi=300, figsize=(6, 3))
plt.plot(t, diff, color="goldenrod")
utkarshGrid()
plt.title("Errors between Bulla model and the emulator")
# plt.ylabel("Fractional Error (Cumulative Magnitude)")
plt.ylabel("Fractional Error (Cumulative Log Flux)")
plt.xlabel("Time (days)")
plt.show()

In [ ]:
# gp = GP5D("Classes/reference.csv")
# gp.split = gpSplit
# gp.emulator = "end"
# gp.cross_validation = (mejdyn, mejwind, phi, iobs)
# gp.set_wv_range(curr_wv)
# gp.n_comp = 25
# time_shape = None
# # gp.save_training_data(time_trauncation = time_shape) # Which wv do you want to extract.  
# gp.save_pca_components(skip_factor = set_skip_factor)
# gp.setXY_cross_validation(mejdyn, mejwind, phi, iobs)
# fitting_kernel = GPy.kern.RBF(input_dim=4, variance = 10, lengthscale=10, ARD = True)
# decay_kernel = GPy.kern.Linear(input_dim=4, ARD = True)
# gp.kernel = fitting_kernel * decay_kernel
# gp.model = GPy.models.GPRegression(gp.X,gp.Y,gp.kernel)
# t0 = time.time()
# print(f"[STATUS] Optimizing...")
# gp.model.optimize_restarts(parallel = True, robust = True, num_restarts = 3)
# print(f"[STATUS] {round(time.time() - t0, 2)}s taken to optimize.")
# # gp.model.optimize(messages = True)
# # display(gp.model.rbf.lengthscale)
# display(gp.model.mul.rbf.lengthscale)
# t1 = time.time()
# gp.model_predict_cross_validation(include_like = True) # Save cross validation
# print(f"[STATUS] {round(time.time() - t1, 3)}s taken to predict single item.")
# gp.model_predict(include_like = True)
# print(f"[STATUS] {round(time.time() - t1, 2)}s taken to predict every item.")
# t2 = time.time()
# gp.save_trained_data()
# print(f"[STATUS] {round(time.time() - t2, 2)}s taken to save data.")
# gp.ComputeDifferenceFlux()
# gp.plot_difference_histogram(binning = 350)
# plt.show()

In [ ]:
# gp.get_flux(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs, time_desired = 2, wv_desired = 950)
# gp.plot_filters(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs)
# gp.overplot_time(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs, wv_desired = 950)
# gp.overplot_wavelength(mejdyn = mejdyn, mejwind =  mejwind, phi = phi, iobs = iobs, time_desired = 2)
# gp.plot_emulator_errors()
# plt.show()